In [1]:
##%%capture
##!pip install mediapipe

In [2]:
##!pip install opencv-python

In [3]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Downloading the data
##!wget http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip

In [5]:
# Loading AFLW2000 Data
#!unzip ./AFLW2000-3D.zip

In [6]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        try:
            results = faces.process(cv2.resize(img[:,:,::-1], dsize=(450,450,3)))
            tdx = int(results.multi_face_landmarks[0].landmark[6].x * img.shape[1])
            tdy = int(results.multi_face_landmarks[0].landmark[6].y * img.shape[0])
        except:
            height, width = img.shape[:2]
            tdx = width / 2
            tdy = height / 2
            

       # height, width = img.shape[:2]
       # tdx = width / 2
       # tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [7]:
# choosing random image
random_file = 'image00058'

faceModule = mediapipe.solutions.face_mesh
# loading image and its correspinding mat file
with faceModule.FaceMesh(static_image_mode=True) as faces:
    # loading the image
    image = cv2.imread('./AFLW2000/'+random_file+'.jpg')
    # processing the face to extract the landmark points (468 point) for each x,y,z
    results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks != None: 
      # looping over the faces in the image
      for face in results.multi_face_landmarks:
          for landmark in face.landmark:
              x = landmark.x
              y = landmark.y
              # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
              shape = image.shape 
              relative_x = int(x * shape[1])
              relative_y = int(y * shape[0])
              # cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
              cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
      cv2.imshow(winname='image',mat=image)
      cv2.waitKey(0)
      cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
image = cv2.imread('./AFLW2000/'+random_file+'.jpg')
# loading the mat file to extract the 3 angels of the face (pitch,yaw,roll)
mat_file = sio.loadmat('./AFLW2000/'+random_file+'.mat')
#extracting the labels 3 angels
pose_para = mat_file["Pose_Para"][0][:3]
pitch = pose_para[0]
yaw = pose_para[1]
roll = pose_para[2]
cv2.imshow("image",draw_axis(image,pitch,yaw,roll))

cv2.waitKey(0)

cv2.destroyAllWindows()

In [9]:
imgs = [Path(f).stem for f in glob.glob("./AFLW2000/*.jpg")]

In [10]:
len(imgs)

2000

In [11]:
# Generating the data matrix from the images using mediapipe

ypr_values = []
x_points = []
y_points = []
with faceModule.FaceMesh(static_image_mode=True) as faces:
    for img in imgs:
        x_points_img = []
        y_points_img = []
        img_arr = cv2.imread("./AFLW2000/" + img + ".jpg")
        img_mat = sio.loadmat("./AFLW2000/" + img + ".mat")
        pose_para = img_mat["Pose_Para"][0][:3]
        pitch = pose_para[0]
        yaw = pose_para[1]
        roll = pose_para[2]
        ypr = [yaw, pitch, roll]
        ypr_values.append(ypr)
        results = faces.process(cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None:
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x_points_img.append(landmark.x)
                    y_points_img.append(landmark.y)
            
        x_points.append(x_points_img)
        y_points.append(y_points_img)    

In [12]:
len(x_points), len(y_points)

(2000, 2000)

In [13]:
len(x_points[0]), len(y_points[0])

(468, 468)

In [14]:
X = np.zeros((2000, 936))

In [15]:
np.array(x_points[0] + y_points[0]).shape

(936,)

In [16]:
for i in range(X.shape[0]):
    try:
        X[i, :] = np.array(x_points[i] + y_points[i])
    except:
        pass

In [17]:
data = pd.DataFrame(data = X, columns = ["feat{}".format(i+1) for i in range(936)])

In [18]:

ypr_values = np.array(ypr_values)

In [19]:
ypr_values.shape

(2000, 3)

In [20]:
data[['yaw', 'pitch', 'roll']] = ypr_values

In [21]:
data
    

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat930,feat931,feat932,feat933,feat934,feat935,feat936,yaw,pitch,roll
0,0.514975,0.535639,0.519629,0.519303,0.539307,0.538134,0.529583,0.385523,0.529236,0.532291,...,0.631829,0.634174,0.504854,0.511276,0.516391,0.492302,0.488564,-0.347654,-0.238109,0.148465
1,0.508752,0.468954,0.493427,0.444709,0.459859,0.455062,0.448820,0.381151,0.432148,0.420305,...,0.636488,0.637607,0.501288,0.511819,0.519926,0.449698,0.435853,0.213283,-0.364073,-0.491296
2,0.523921,0.500583,0.510641,0.468734,0.494248,0.487617,0.473420,0.347366,0.460440,0.452428,...,0.581764,0.589793,0.472434,0.477826,0.479639,0.442669,0.431431,0.125325,0.117923,-0.295807
3,0.503006,0.494408,0.499270,0.476528,0.493112,0.492547,0.492994,0.362106,0.492332,0.491318,...,0.597537,0.602776,0.483326,0.488594,0.491245,0.474374,0.466441,0.057434,0.151753,-0.052361
4,0.511498,0.490996,0.503665,0.472855,0.487056,0.486188,0.488955,0.366738,0.484404,0.479625,...,0.646516,0.649855,0.493819,0.500156,0.505251,0.466047,0.453510,0.194847,-0.126741,-0.149632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.468223,0.420989,0.461647,0.423274,0.414745,0.420752,0.446074,0.418327,0.445505,0.437089,...,0.610119,0.612023,0.458667,0.469096,0.477510,0.425301,0.412821,0.943137,-0.339877,-0.492823
1996,0.565490,0.602843,0.558723,0.566782,0.605669,0.592987,0.548913,0.417403,0.537847,0.541224,...,0.618065,0.623245,0.496907,0.504872,0.510608,0.480970,0.472296,-0.919017,-0.101006,-0.099479
1997,0.564053,0.570461,0.554711,0.524573,0.566011,0.552306,0.511909,0.362014,0.491613,0.485351,...,0.585393,0.591526,0.497967,0.501860,0.503224,0.475402,0.468079,-0.527796,0.206154,-0.310925
1998,0.547263,0.603300,0.557967,0.595140,0.612713,0.611146,0.596982,0.487875,0.606692,0.618657,...,0.672409,0.670938,0.550959,0.557638,0.565010,0.548018,0.539153,-1.028589,-1.083322,1.162738


In [22]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

In [23]:
from xgboost import XGBRFRegressor

In [24]:
from sklearn.decomposition import PCA

In [25]:
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
features = data.drop(columns=['yaw', 'pitch', 'roll']).values
targets = data[['yaw', 'pitch', 'roll']].values

### Support Vector Machines baseline Model

In [27]:
x_train, x_val, y_train, y_val = train_test_split(features, targets, random_state=42, test_size=0.1, shuffle=True)

In [28]:
x_train.shape, y_train.shape

((1800, 936), (1800, 3))

In [29]:
svm_model = SVR()

In [30]:
multi_1 = MultiOutputRegressor(svm_model)

In [31]:
multi_1.fit(x_train, y_train)

MultiOutputRegressor(estimator=SVR())

In [32]:
preds_svm = multi_1.predict(x_val)

In [33]:
from sklearn.metrics import mean_squared_error, r2_score

In [34]:
mean_squared_error(y_val, preds_svm)

0.2554631464915783

In [35]:
r2_score(y_val, preds_svm)

0.32031561628079525

### XGBOOST baseline model

In [36]:
xg_model = XGBRFRegressor()

In [37]:
multi_2 = MultiOutputRegressor(xg_model)

In [38]:
multi_2.fit(x_train, y_train)

MultiOutputRegressor(estimator=XGBRFRegressor(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              max_bin=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              objective='reg:squarederror',
                                              predictor=None, random_state=None,
                                              reg_alpha=None,
                                              sampling_method=None,
                                              scale_pos_weight=None, ...))

In [39]:
preds_xg = multi_2.predict(x_val)

In [40]:
mean_squared_error(y_val, preds_xg)

0.25908417101943865

In [41]:
r2_score(y_val, preds_xg)

0.3115044294570222

### Random Forest baseline model

In [42]:
rf_model = RandomForestRegressor()

In [43]:
rf_model.fit(x_train, y_train)

RandomForestRegressor()

In [44]:
preds_rf = rf_model.predict(x_val)

In [45]:
mean_squared_error(y_val, preds_rf)

0.2530566012311036

In [46]:
r2_score(y_val, preds_rf)

0.3283332554752705

### AdaBoost baseline model

In [47]:
ada_model = AdaBoostRegressor()

In [48]:
multi_3 = MultiOutputRegressor(ada_model)

In [49]:
multi_3.fit(x_train, y_train)

MultiOutputRegressor(estimator=AdaBoostRegressor())

In [50]:
preds_ada = multi_3.predict(x_val)

In [51]:
mean_squared_error(y_val, preds_ada)

0.27306479336321515

In [52]:
r2_score(y_val, preds_ada)

0.2750754313696914

### testing on video

In [53]:
cap = cv2.VideoCapture('./test_video2.mp4')

if (cap.isOpened()== False): 
  print("Error opening video  file")

#see number of frames
int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

318

In [54]:
faceModule = mediapipe.solutions.face_mesh

faces = faceModule.FaceMesh(static_image_mode=True)

In [55]:
#set first dimension as number of frames
test_Data = np.zeros((318,936))

In [56]:
x_points = []
y_points = []
with faceModule.FaceMesh(static_image_mode=True) as faces:
    for i in range(test_Data.shape[0]):
        ret, frame = cap.read()
        if ret == True:
            results = faces.process(frame[:,:,::-1])
        x_points_img = []
        y_points_img = []
        if results.multi_face_landmarks is not None:
            
            for face in results.multi_face_landmarks:
                
                for landmark in face.landmark:
                    
                    x_points_img.append(landmark.x)
                    y_points_img.append(landmark.y)
        x_points.append(x_points_img)
        y_points.append(y_points_img)

In [57]:
len(x_points)

318

In [58]:
for i in range(len(x_points)):
    test_Data[i, :] = np.array(x_points[i] + y_points[i])

In [59]:
test_Data.shape

(318, 936)

In [60]:
test_df = pd.DataFrame(data=test_Data)

In [61]:
test_Data[test_df.isna() == True]

array([], dtype=float64)

In [62]:
cap = cv2.VideoCapture('./test_video2.mp4')

if (cap.isOpened()== False): 
  print("Error opening video  file")

In [63]:
img_array = []
for i in range(test_Data.shape[0]):
    ret, frame = cap.read()
    if ret == True:
        print(i)
        frame = draw_axis(frame,
                        yaw=multi_1.predict(test_Data[i].reshape(1,-1))[0][0],
                        pitch=multi_1.predict(test_Data[i].reshape(1,-1))[0][1],
                        roll=multi_1.predict(test_Data[i].reshape(1,-1))[0][2])
        img_array.append(frame)
        height, width, layers = frame.shape
        size  = (width,height)
    else:
        
        break

out = cv2.VideoWriter('output_video1.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
    print(i)
out.release()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317


In [64]:
cap.release()
cv2.destroyAllWindows()
test_Data.shape

(318, 936)